# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

The purpose of this excercice is to find the safest and most secure location for opening of commercial establishments in Baltimore, US. 
This report will be targeted to stakeholders interested in opening any business like **convinience Store** in **Baltimore City**, US.


This analysis will require data science tools to analyse data and identify the safest districts and neighborhoods as a business oportunity and avoid the dangerous ones.

Based on definition of our problem, the factor that will influence our decission is: The number of crimes by neighorhood/


We will be using the geographical coordinates to plot neighbourhoods and finally cluster them with the analysis of our findinds

Following data sources will be needed to extract/generate the required information:

- [**Part 1**: Using a real world data set from wwww.data.world/com containing the baltimore Crimes from 2011 to 2016](#part1): 

 A dataset consisting of the crime statistics of each Neighbourhood along with crime date, crime time, crime code, location of the crime. description of the crime, weapon used, police post engaged, district and neighborhood where the crime took place, the geo-localisation (coordinates) of the crime scene and how many crimes were commited in each neighborhood.

- [**Part 2**: Transforming the Data into a dataframe ](#part2):


- [**Part 3**: Exploring and Visualzing the Data](#part3): 



- [**Part 4**: Mapping the identified districts and neighborhoods.](#part4): 

###**Part 1:** Using a real world data set from wwww.data.world/com containing the baltimore Crimes from 2011 to 2016<a name="part1"></a>


####  Baltimore Crime data 

Properties of the Crime Report

*   crimedate: date
*   crimetime: string
*   crimecode: string
*   location: string
*   description: string
*   weapon: string
*   post: integer
*   district: string
*   neighborhood: string
*   location_1: geopoint
*   total_incidents: boolean



Data set URL: https://data.world/baltimore/baltimore-crime-data

### Importing all the necessary Libraries

In [13]:
### Importing all the necessary Libraries

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import plotly.express as px #Library to plot data
import folium #Library to show data on map
import re #Library to parse data
from sklearn.cluster import KMeans #Library to cluster data
import random # library for random number generation
import matplotlib.pyplot as plt #Library to plot data
%matplotlib inline


print('Libraries imported.')

Libraries imported.


[link text](https://)###Reading from the Dataset



In [14]:
df = pd.read_csv(r'C:\Users\jadya\Downloads\BPD_Part_1_Victim_Based_Crime_Data.csv')

In [15]:
#Identifying the empty cells
df.isnull().sum()

CrimeDate               0
CrimeTime               0
CrimeCode               0
Location                2
Description          4003
Weapon             173375
Post                   13
District                0
Neighborhood            0
Location 1              0
Total Incidents         0
dtype: int64

In [16]:
# A glimpe into the dataset
df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,Post,District,Neighborhood,Location 1,Total Incidents
0,6/18/2016,0:33:00,4E,2700 CHESLEY AVE,I,HANDS,424.0,NORTHEASTERN,North Harford Road,"(39.3679000000, -76.5555900000)",1
1,6/18/2016,0:39:00,4B,2700 FAIT AVE,O,KNIFE,232.0,SOUTHEASTERN,Canton,"(39.2831500000, -76.5783400000)",1
2,6/18/2016,15,9S,2400 CYLBURN AV,Outside,FIREARM,532.0,NORTHERN,Levindale,"(39.3510400000, -76.6597600000)",1
3,6/18/2016,1:53:00,3AF,2300 ORLEANS ST,O,FIREARM,221.0,SOUTHEASTERN,McElderry Park,"(39.2955600000, -76.5844600000)",1
4,6/18/2016,2:05:00,6C,800 N WOLFE ST,I,NaN,321.0,EASTERN,Middle East,"(39.3002700000, -76.5909700000)",1


###**Part 2:**  Transforming the Data into a dataframe <a name="part2"></a>


In [17]:
# Extracting the month and year of each crime for better effectiveness
df['Year'] = pd.DatetimeIndex(df['CrimeDate']).year
df['Month'] = pd.DatetimeIndex(df['CrimeDate']).month

In [18]:
#Dropping unnecessary columns
df.drop(df[['CrimeDate','CrimeTime']], axis=1, inplace=True)
df.head()

,CrimeCode,Location,Description,Weapon,Post,District,Neighborhood,Location 1,Total Incidents,Year,Month
0,4E,2700 CHESLEY AVE,I,HANDS,424.0,NORTHEASTERN,North Harford Road,"(39.3679000000, -76.5555900000)",1,2016,6
1,4B,2700 FAIT AVE,O,KNIFE,232.0,SOUTHEASTERN,Canton,"(39.2831500000, -76.5783400000)",1,2016,6
2,9S,2400 CYLBURN AV,Outside,FIREARM,532.0,NORTHERN,Levindale,"(39.3510400000, -76.6597600000)",1,2016,6
3,3AF,2300 ORLEANS ST,O,FIREARM,221.0,SOUTHEASTERN,McElderry Park,"(39.2955600000, -76.5844600000)",1,2016,6
4,6C,800 N WOLFE ST,I,NaN,321.0,EASTERN,Middle East,"(39.3002700000, -76.5909700000)",1,2016,6


In [19]:
#Exracting latitude and longitude data for further use:
 
Latitude=[]
Longitude=[]
for i in range(len(df)):
    a=re.split('[(),]',df['Location 1'][i])[1]
    b=re.split('[(),]',df['Location 1'][i])[2]
    Latitude.append(a)
    Longitude.append(b)

In [20]:
#Converting the extracted data into data_frames:

df_lat=pd.DataFrame(Latitude,columns=['Latitude'])
df_long=pd.DataFrame(Longitude, columns=['Longitude'])

In [21]:
#Merging the dataframes with longitude and latitude data to the original dataframe:

df_lat.head().index
df1=df.join(df_lat,how='outer')
df_map=df1.join(df_long, how='outer')

In [22]:
#The final version of the dataframe

df_map.head()

,CrimeCode,Location,Description,Weapon,Post,District,Neighborhood,Location 1,Total Incidents,Year,Month,Latitude,Longitude
0,4E,2700 CHESLEY AVE,I,HANDS,424.0,NORTHEASTERN,North Harford Road,"(39.3679000000, -76.5555900000)",1,2016,6,39.3679000000,-76.5555900000
1,4B,2700 FAIT AVE,O,KNIFE,232.0,SOUTHEASTERN,Canton,"(39.2831500000, -76.5783400000)",1,2016,6,39.2831500000,-76.5783400000
2,9S,2400 CYLBURN AV,Outside,FIREARM,532.0,NORTHERN,Levindale,"(39.3510400000, -76.6597600000)",1,2016,6,39.3510400000,-76.6597600000
3,3AF,2300 ORLEANS ST,O,FIREARM,221.0,SOUTHEASTERN,McElderry Park,"(39.2955600000, -76.5844600000)",1,2016,6,39.2955600000,-76.5844600000
4,6C,800 N WOLFE ST,I,NaN,321.0,EASTERN,Middle East,"(39.3002700000, -76.5909700000)",1,2016,6,39.3002700000,-76.5909700000


###**Part 3:**  Exploring and Visualzing the Data <a name="part3"></a>

##Methodology<a name="methodology"></a>

Categorized the methodologysection into two parts:

- [**Exploratory Data Analysis**:](#eda) Visualise the crime repots in different Vancouver boroughs to idenity the safest borough and normalise the neighborhoods of that borough. We will Use the resulting data and find 10 most common venues in each neighborhood.


- [**Modelling**:](#mdl) To help stakeholders choose the right neighborhood within a borough we will be clustering similar neighborhoods using K - means clustering which is a form of unsupervised machine learning algorithm that clusters data based on predefined cluster size. We will use K-Means clustering to address this problem so as to group data based on existing venues which will help in the decision making process.

####Exploratory Data Analysis<a name="eda"></a>

In [23]:
# Extracting the total incidents by month:

Incidents_by_month =df.groupby(['Month'])['Total Incidents'].sum().reset_index()
Incidents_by_month.head()

,Month,Total Incidents
0,1,21580
1,2,17928
2,3,22402
3,4,23382
4,5,26126


In [24]:
# Plotting the data: 
fig = px.bar(Incidents_by_month , x="Month", y="Total Incidents", title='Total number of crimes by month',color='Total Incidents')
fig.show()

In [25]:
# Extracting the weapons used in the incidents:
bar_data=df.groupby(['Weapon'])['Total Incidents'].sum().reset_index()
bar_data.sort_values(by='Total Incidents', ascending=False, na_position='first', inplace=True)
bar_data.head()

,Weapon,Total Incidents
1,HANDS,47885
0,FIREARM,18960
3,OTHER,13226
2,KNIFE,9177


In [26]:
# Plotting the data: 
fig = px.bar(bar_data, x="Weapon", y="Total Incidents", title='Weapon used most in the crimes', height=400, width=600, color='Weapon')
fig.show()


In [27]:
#Identifying the most dangerous and most peacefull districts and neighborhoods:

Incidents_by_neighborhood=df.groupby(['District','Neighborhood'])['Total Incidents'].sum().reset_index()
Incidents_by_neighborhood.sort_values(by='Total Incidents', ascending=False, na_position='first', inplace=True)
Incidents_by_neighborhood = Incidents_by_neighborhood.reset_index(drop=True)
Most_dangerous_neighborhoods=Incidents_by_neighborhood.head()
safest_neighborhoods=Incidents_by_neighborhood.tail()

In [78]:
#Creating a dataframe for the top 5 dangerous districts:

District_name=[]
Number_of_crimes=[]

for i in range(len(df_map['District'].unique())):
    District= df_map['District'].unique()[i]
    total = df_map.loc[df_map['District'] == District, 'Total Incidents'].sum()
    District_name.append(District)
    Number_of_crimes.append(total)

    df_District_name=pd.DataFrame(District_name,columns=['District_name'])
    df_Number_of_crimes=pd.DataFrame(Number_of_crimes,columns=['Number_of_crimes'])
    df_District_name=df_District_name.join(df_Number_of_crimes,how='outer')
    df_District_name.sort_values(by='Number_of_crimes', ascending=False, na_position='first', inplace=True)
    df_most_dangerous_districts=df_District_name.head()

In [79]:
df_most_dangerous_districts

,District_name,Number_of_crimes
0,NORTHEASTERN,41906
1,SOUTHEASTERN,36425
5,CENTRAL,30975
4,SOUTHERN,30161
2,NORTHERN,29310


In [83]:
# Plotting the data for the most dangerous districts: 


fig = px.bar(df_most_dangerous_districts, x="District_name", y="Number_of_crimes", title='Most_dangerous_Districts', height=400, width=600, color='District_name')
fig.show()

In [85]:
# Plotting the data for the most dangerous neighborhoods: 

fig = px.bar(Most_dangerous_neighborhoods, x="Neighborhood", y="Total Incidents", title='Top_5_Most_dangerous_neighborhoods', height=400, width=600, color='Neighborhood')
fig.show()

In [86]:
#Creating a dataframe for the top 5 safest districts:

Safest_dist=df_District_name.tail()

In [87]:
# Plotting the data for the least dangerous districts: 

fig = px.bar(Safest_dist, x="District_name", y="Number_of_crimes", title='Most_dangerous_Districts', height=400, width=600, color='District_name')
fig.show()

In [32]:
# Plotting the data for the least dangerous neighborhoods: 

fig = px.bar(safest_neighborhoods, x="Neighborhood", y="Total Incidents", title='Top_5_Safest_neighborhoods', height=400, width=600, color='Neighborhood')
fig.show()

### Based on exploratory data the western district has the lowest crimes number and northeastern is the most dangerous one on the other hand.

##### The top 5 safest neighborhoods are: Gwynns falls/Leaking park, Penn North, Penn-Fallsway, Bridgeview/Greenlawn and Franklin square
##### The most dangerous neighborhoods by order are Downtown, Frankford, Belair-Edision, Brooklyn, Cherry Hill   

####Modelling<a name="#mdl"></a>

To help stakeholders choose the right neighborhoods within a district we will be clustering similar neighborhoods using K - means clustering which is a form of unsupervised machine learning algorithm that clusters data based on predefined cluster size.

The model Kmeans is selected for this excercice as it allows a clear classification of the districts and neighborhoods which alignes with the purpose of the analysis.

###**Part 4:** Mapping the identified districts and neighborhoods <a name="part4"></a>

Mapping the identified districts and neighborhoods for the last year of the data_set: 2016 (being the most updated input and to falicitate the excursion of the model)

##The top 30 of the most dangerous districs and neighborhoods:

In [33]:
# Selecting the corresponding data and feeding it to a map: 

Dangerous_Nei=df_map.groupby(['Neighborhood','Longitude','Latitude','District','Year'])['Total Incidents'].sum().reset_index()
Dangerous_Nei.sort_values(by='Total Incidents', ascending=False, na_position='first', inplace=True)
Dangerous_Nei.reset_index(drop=True)
Dangerous_Neighborhoods=Dangerous_Nei[:30]


map_Baltimore = folium.Map(location=[39.3679000000,-76.5555900000],zoom_start=10)
for lat,lng,Neighborhood,District in zip(Dangerous_Neighborhoods['Latitude'],Dangerous_Neighborhoods['Longitude'],Dangerous_Neighborhoods['Neighborhood'],Dangerous_Neighborhoods['District']):
    label = '{}'.format(Neighborhood)
    folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='Maroon',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.3,
    parse_html=False).add_to(map_Baltimore)
map_Baltimore

The top 30 safestet districs and neighborhoods

In [93]:
# Selecting the corresponding data and feeding it to a map: 


Most_peaceful_Nei=df_map.groupby(['Neighborhood','Longitude','Latitude','District'])['Total Incidents'].sum().reset_index()
Most_peaceful_Nei.sort_values(by='Total Incidents', ascending=True, na_position='first', inplace=True)
Most_peaceful_Nei.reset_index(drop=True)
Most_peaceful_Neighborhoods=Most_peaceful_Nei[:30]


map_Baltimore = folium.Map(location=[39.3456,-76.5734],zoom_start=10)
for lat,lng,Neighborhood,District in zip(Most_peaceful_Neighborhoods['Latitude'],Most_peaceful_Neighborhoods['Longitude'],Most_peaceful_Neighborhoods['Neighborhood'],Most_peaceful_Neighborhoods['District']):
    label = '{},{}'.format(Neighborhood,District)
    folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='Green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Baltimore)
#map_Baltimore

In [35]:
Dangerous_Neighborhoods.head()

,Neighborhood,Longitude,Latitude,District,Year,Total Incidents
83480,Inner Harbor,-76.6130900000,39.2850000000,CENTRAL,2012,204
101343,Mondawmin,-76.6582100000,39.3180000000,NORTHWESTERN,2011,170
25394,Carroll - Camden Industrial Area,-76.6276900000,39.2740800000,SOUTHERN,2015,139
101347,Mondawmin,-76.6582100000,39.3180000000,NORTHWESTERN,2015,135
35004,Cherry Hill,-76.6358400000,39.2466100000,SOUTHERN,2016,131


In [36]:
#Selecting the data_segment:

Dangerous_N=df_map.groupby(['Neighborhood','Longitude','Latitude','District','Year'])['Total Incidents'].sum().reset_index()
Dangerous_N.sort_values(by='Total Incidents', ascending=False, na_position='first', inplace=True)
Dangerous_N.reset_index(drop=True)
Dangerous_Neighborhoods_clusters=Dangerous_N[Dangerous_N['Year']==2016]

In [37]:
##Clustering Neighbourhoods:

k=5
Baltimore_clustering = Dangerous_Neighborhoods_clusters.drop(['Neighborhood','District'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(Baltimore_clustering)
kmeans.labels_
Dangerous_Neighborhoods_clusters.insert(0, 'Cluster Labels', kmeans.labels_)

In [38]:
## Analysis and Examining the resulting Clusters <a name="analysis"></a>Cluster Labels

In [39]:
# create map
map_clusters = folium.Map(location=[39.3679000000,-76.5555900000],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(Dangerous_Neighborhoods_clusters['Latitude'], Dangerous_Neighborhoods_clusters['Longitude'], Dangerous_Neighborhoods_clusters['Neighborhood'], Dangerous_Neighborhoods_clusters['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)

In [101]:
#map_clusters

In [41]:

## Results and Discussion <a name="results"></a>

The purpose of the business problem was to help stakeholders identify the safest districts  in Baltimore, and an appropriate neighborhood within it for a proximity commercial business opportunity.

This has been achieved by making use of Baltimore crime data to identify a safe district with considerable number of neighborhoods for any business to be viable.
This excercice helped in getting to know better the targeted market in terms of safety by identifying the months with most of the incidents to take their necessary measures proactively especially taking into consideration that most of the crimes were not commited using a weapon. 

We achieved this by grouping the neighborhoods into clusters to assist the stakeholders by providing them with relavent data about safety of a given neighborhood if they have an interest to invest in a particular neighborhood outside of the top safest ones selected.

## Conclusion <a name="conclusion"></a>

We have explored the crime data to understand different types of crimes, the weapons used in them as well as the months with most incidents in all districts of Baltimore and later categorized them into different neighborhoods to help stakeholders shortlist their target. 